In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.externals import joblib

from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold

import sys
sys.path.insert(0, 'G:/work/GitHub/ml_baseline_things/functions/')
import GridSearchCVcustom as GSC
import featureEngineringFunctions as fef
from sklearn.preprocessing import StandardScaler

dirPath = '2-featuresPack/'

In [2]:
#featuresFromOtherSolutions = ['Survived','Pclass','Sex','Family_Size','Family_Survival','Fare_Bin','Age_Bin']

featureList = [
#'PassengerId',
#'Survived',
#'AGE_cont',
'AgeBin_LE',
#'Age-Class_cont',
#'Deck_LE',
#'Deck_OHE',
'FamilySize_LE',
#'Fare_cont',
'FareBin_LE',
#'HasBloodRelatives_LE',
#'HasWifeHasb_LE',
#'IsAlone_LE',
#'Parch_LE',
'Pclass_LE',
'Sex_LE',
#'Sex_OHE',
#'SibSp_LE',
#'Title_LE',
#'Title_OHE',
'FamilySurvival_bin'
]
#features = ['Survived','Pclass','Sex','Family_Size','Family_Survival','Fare_Bin','Age_Bin']

trainVStest = 891
dataAll = fef.create_dataset_from_features (featureList, dirPath)
dataTrain = fef.create_dataset_from_features (featureList, dirPath)[:trainVStest]
dataTrainTarget =fef.create_dataset_from_features (['Survived'], dirPath)[:trainVStest]

dataTrainScaled = StandardScaler().fit(dataAll).transform(dataTrain)

In [3]:
#check available params
pipeline = GSC.get_pipeline_1step(xgb.XGBClassifier(n_jobs=-1))
pipeline.named_steps 

{'clf': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
        n_jobs=-1, nthread=None, objective='binary:logistic',
        random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
        seed=None, silent=True, subsample=1)}

In [9]:
RS = 100
score = "roc_auc"
n_splits = 5
nSamples = len(dataTrain)
cvShuffle = True
paramGrid = {'clf': [xgb.XGBClassifier(n_jobs=-1)],
              'clf__random_state': [RS],
              'clf__class_weight': ['balanced'],
              'clf__n_estimators': [10, 100, 250, 500, 750, 1000],
              'clf__max_depth': [1, 2, 4],
              'clf__learning_rate': [0.01, 0.02, 0.05, 0.1, 0.5]
             }
cv = StratifiedKFold(n_splits=n_splits, random_state=RS, shuffle=cvShuffle)

# non-Scaled + StratifiedKFold

In [5]:
bestScore1, bestScoreParams1, bestIter1 = GSC.GridSearchCVcustom(pipeline, paramGrid, 
                                                              dataTrain, dataTrainTarget,
                                                              cv=cv, n_jobs = -1, scoring = score, pre_dispatch=2,
                                                              showSteps=True, itersToPerform ='All')
print ('bestScore = {}\nBestScoreParams = {}'.format(bestScore1, bestScoreParams1))

Total iterations exist for provided gridSearch params: 90
These steps available to be performed:
v Iter 1: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 10  clf__random_state: 100  
v Iter 2: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 100  clf__random_state: 100  
v Iter 3: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 250  clf__random_state: 100  
v Iter 4: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 500  clf__random_state: 100  
v Iter 5: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 750  clf__random_state: 100  
v Iter 6: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 1000  clf__random_state: 100  
v Iter 7: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 2  clf__n_estimators: 10

New best result = 0.76681 (std = 0.02215) during 15.0 [secs] (iter = 1) for search with params 
dict_items([('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)), ('clf__class_weight', 'balanced'), ('clf__learning_rate', 0.01), ('clf__max_depth', 1), ('clf__n_estimators', 10), ('clf__random_state', 100)])

New best result = 0.83729 (std = 0.02636) during 8.0 [secs] (iter = 2) for search with params 
dict_items([('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, n

Iterations to perform: 11
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.1

Iterations to perform: 6
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.1

Iterations to perform: 1
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.0

Congrats! All DONE. Total time is [mins]: 1.19
BestIter was 50, bestScore = 0.8874091492128635, bestScoreParams = {'clf': XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
       colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=100,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)}
bestScore = 0.8874091492128635
BestScoreParams = {'clf': XGBClassifier(base_score=0.5, booster='gbtree', cla

# Scaled + StratifiedKFold

In [6]:
bestScore2, bestScoreParams2, bestIter2 = GSC.GridSearchCVcustom(pipeline, paramGrid, 
                                                              dataTrainScaled, dataTrainTarget,
                                                              cv=cv, n_jobs = -1, scoring = score, pre_dispatch=2,
                                                              showSteps=True, itersToPerform ='All')
print ('bestScore = {}\nBestScoreParams = {}'.format(bestScore2, bestScoreParams2))

Total iterations exist for provided gridSearch params: 90
These steps available to be performed:
v Iter 1: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 10  clf__random_state: 100  
v Iter 2: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 100  clf__random_state: 100  
v Iter 3: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 250  clf__random_state: 100  
v Iter 4: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 500  clf__random_state: 100  
v Iter 5: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 750  clf__random_state: 100  
v Iter 6: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 1  clf__n_estimators: 1000  clf__random_state: 100  
v Iter 7: clf__class_weight: balanced  clf__learning_rate: 0.01  clf__max_depth: 2  clf__n_estimators: 10

New best result = 0.76681 (std = 0.02215) during 0.0 [secs] (iter = 1) for search with params 
dict_items([('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)), ('clf__class_weight', 'balanced'), ('clf__learning_rate', 0.01), ('clf__max_depth', 1), ('clf__n_estimators', 10), ('clf__random_state', 100)])

New best result = 0.83729 (std = 0.02636) during 0.0 [secs] (iter = 2) for search with params 
dict_items([('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nt

Iterations to perform: 11
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.1

Iterations to perform: 6
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.0

Iterations to perform: 1
Average time for iteration [mins]: 0.01
Job will be done in (approximately) [mins] : 0.0

Congrats! All DONE. Total time is [mins]: 0.81
BestIter was 50, bestScore = 0.8874091492128635, bestScoreParams = {'clf': XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
       colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=100,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)}
bestScore = 0.8874091492128635
BestScoreParams = {'clf': XGBClassifier(base_score=0.5, booster='gbtree', cla

## Pure GS

In [12]:
clf = xgb.XGBClassifier(n_jobs=-1)

params = {
              'random_state': [RS],
              #'class_weight': ['balanced'],
              'n_estimators': [10, 100, 250, 500, 750, 1000],
              'max_depth': [1, 2, 4],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.5]
             }

# Using ROC_AUC as metric has a better result than using accuracy. 
gs = GridSearchCV(clf, param_grid= params, cv = cv, n_jobs = -1 ,scoring = score,verbose=2)
gs.fit(dataTrainScaled, dataTrainTarget)
print(gs.best_score_)
print(gs.best_estimator_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   59.9s


0.8874134718911294
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=100, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  1.3min finished


In [15]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=100, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [14]:
cv

StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

## save to disk

In [16]:
#model = bestScoreParams2['clf']
model = gs.best_estimator_
model.fit(dataTrainScaled, dataTrainTarget)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=100, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [17]:
@comment_this_line_if_ready
from datetime import datetime
_ = joblib.dump([model, featureList], 
            open('modelsTrained/modelXGB--{}.pkl'.format(datetime.now().strftime("%Y-%m-%d--%H-%M")), 'wb'), 9)